## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

The Starter Code suggested to keep 8 application types and bin the rest into "Other". I noticed that the application T3 appears 27K times while the rest appears maxim 1.5K times. I decided to only create 2 bins : T3 and other.

In [6]:
# Replace in dataframe
for app in application_df['APPLICATION_TYPE']:
  if app != "T3":
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     7262
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

I used the same binning logic for the classification column. This way, there's a sort of balance inside my data (a boolean balance).

In [8]:

#I used a different code then the provided one. Using an if statement to change the data if it's not in in the list created above
for i in application_df['CLASSIFICATION'] :
  if i != 'C1000':
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(i,"Other")
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
Other    16973
Name: count, dtype: int64

My second opinion was regarding the income amount. I first ran a code to see the number of organizations per income amount

In [9]:
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

Looking at the above layout, I decided to bin this differently.
0, 1 to 5M and 5M+. You'll notice the cell is commented out and that is explained later, during our model training.

In [10]:
#I used a different code then the provided one. Using an if statement to change the data if it's not in in the list created above
#for i in application_df['INCOME_AMT'] :
#  if (i == '1-9999') | (i == '10000-24999') | (i == '25000-99999') | (i == '100000-499999') | (i == '1M-5M'):
  #  application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(i,"1 to 5M")
 # elif i != '0':
 #   application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(i,"5M+")
# Check to make sure binning was successful
#application_df['INCOME_AMT'].value_counts()

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
#colums to transform APPLICATION_TYPE, AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,SPECIAL_CONSIDERATIONS,INCOME_AMT
application_type_dummies = pd.get_dummies(application_df['APPLICATION_TYPE'], dtype=int)
income_amt_dummies = pd.get_dummies(application_df['INCOME_AMT'], dtype=int)
affiliation_dummies = pd.get_dummies(application_df['AFFILIATION'], dtype=int)
classification_dummies = pd.get_dummies(application_df['CLASSIFICATION'], dtype=int)
use_case_dummies = pd.get_dummies(application_df['USE_CASE'], dtype=int)
organization_dummies = pd.get_dummies(application_df['ORGANIZATION'], dtype=int)
special_considerations_dummies = pd.get_dummies(application_df['SPECIAL_CONSIDERATIONS'], dtype=int)
application_df_dummies = application_df.drop(['APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION',
                                              'USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS','INCOME_AMT'], axis = 1)
application_df_dummies = pd.concat([application_df_dummies,application_type_dummies,
                                   affiliation_dummies,classification_dummies,organization_dummies,
                                   special_considerations_dummies,income_amt_dummies], axis = 1 )

pd.set_option('display.max_columns', None)
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,Other,T3,CompanySponsored,Family/Parent,Independent,National,Other,Regional,C1000,Other,Association,Co-operative,Corporation,Trust,N,Y,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,1,5000,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0


Now that we have combined some features into one, we have less columns but more weight on each of these columns. The rest of the optimization will occur in the model creation

In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies['IS_SUCCESSFUL']
X = application_df_dummies.drop(['IS_SUCCESSFUL'], axis = 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

* Before changing my model, I chose to ran it as it was, just to check the result from the features adjustment done above (keep 2 hidden layers with tanh activations and 1 oupt layer with a sigmoid activation + 100 epochs). It returned an accuracy of 69% (3% less then my initial model). Mathematically speaking, I believe this is normal as we lowered the number of our explanatory variables, going agains the Law of Large Numbers. However, I believe that we could help by adding another layer and increassing our epoch.
* Now, I decided to compensate with this "loss" by adding an extra layer and increasing each nod by 10. Unfortunately for me. The accuracy remained at 69%.
* My next try was to run 250 epochs instead of 100. We stayed at an accuracy of 69%
* My last try was to maybe remove the binning done for the income amt. The accuracy reached 70%.

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=90,activation = 'relu', input_dim = len(X.columns)))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=40,activation = 'relu'))

#Thirs hidden layer
nn2.add(tf.keras.layers.Dense(units=35,activation = 'relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1,activation = 'sigmoid')) #as we want a result between 0 and 1

# Check the structure of the model
nn2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                2520      
                                                                 
 dense_1 (Dense)             (None, 40)                3640      
                                                                 
 dense_2 (Dense)             (None, 35)                1435      
                                                                 
 dense_3 (Dense)             (None, 1)                 36        
                                                                 
Total params: 7631 (29.81 KB)
Trainable params: 7631 (29.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs= 500)

Epoch 1/500
804/804 [==============================] - 3s 2ms/step - loss: 0.6058 - accuracy: 0.6925
Epoch 2/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5994 - accuracy: 0.6969
Epoch 3/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5970 - accuracy: 0.7006
Epoch 4/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5976 - accuracy: 0.6984
Epoch 5/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5977 - accuracy: 0.6997
Epoch 6/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5969 - accuracy: 0.7005
Epoch 7/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5963 - accuracy: 0.7002
Epoch 8/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5965 - accuracy: 0.6997
Epoch 9/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5963 - accuracy: 0.7014
Epoch 10/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5960 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6263 - accuracy: 0.6999 - 533ms/epoch - 2ms/step
Loss: 0.626312792301178, Accuracy: 0.6999416947364807


In [18]:
# Export our model to HDF5 file
#nn2.save('AlphabelSoupCharity_Optimization.h5')